## **IMPORTAMOS LAS LIBRERIAS**

In [21]:
! pip install pyodbc 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyodbc #importamos 
import os
import warnings

warnings.filterwarnings("ignore")


---

## **ETL**

#### explicamos
- 1
- 1

In [ ]:
# Descargamos los datos del KAGGLE
path = kagglehub.dataset_download("wyattowalsh/basketball")

# Ruta de la carpeta que contiene los archivos CSV
path = path.replace("\\", "/")  # Cambiar barras invertidas
carpeta = fr"{path}/csv"

# Lista de archivos específicos a usar
archivos_a_usar = ["game.csv", "inactive_players.csv", "play_by_play.csv", "team.csv", "common_player_info.csv", "draft_combine_stats.csv", "other_stats.csv"]

# Diccionario para almacenar los DataFrames con nombres clave
dataframes = {}

# Parámetros para optimización
chunksize = 10000  # Leer en fragmentos de 10,000 filas
columnas_necesarias = None  # Especifica columnas necesarias, si las conoces (lista de nombres)
dtypes = None  # Especifica tipos de datos, si los conoces (diccionario)

# Iterar sobre los archivos en la carpeta y filtrar según la lista
for archivo in os.listdir(carpeta):
    if archivo in archivos_a_usar:  # Verifica si el archivo está en la lista
        ruta_archivo = os.path.join(carpeta, archivo)
        try:
            # Variable temporal para almacenar los fragmentos del archivo
            fragmentos = []
            
            # Leer el archivo en fragmentos (chunks)
            for chunk in pd.read_csv(
                ruta_archivo, 
                chunksize=chunksize, 
                usecols=columnas_necesarias,  # Especificar columnas necesarias (si corresponde)
                dtype=dtypes  # Especificar tipos de datos para optimizar memoria (si corresponde)
            ):
                # Opcional: Procesar cada fragmento aquí antes de agregarlo
                fragmentos.append(chunk)

            # Combinar los fragmentos en un solo DataFrame para este archivo
            df = pd.concat(fragmentos, ignore_index=True)
            
            # Asignar al diccionario usando el nombre del archivo sin extensión como clave
            nombre_variable = os.path.splitext(archivo)[0]  # Quitar la extensión
            dataframes[nombre_variable] = df
            
            print(f"Archivo leído correctamente: {archivo}")
        except Exception as e:
            print(f"Error al leer el archivo {archivo}: {e}")

# Acceso individual a los DataFrames
# Tabla de games
games = dataframes.get("game")

# Tabla de inactivos
inactive_players = dataframes.get("inactive_players")

# Tabla de resumen general
game_detail = dataframes.get("play_by_play")

# Tabla de equipos
teams = dataframes.get("team")

#Informacion de jugadores
info_players = dataframes.get("common_player_info")



Archivo leído correctamente: common_player_info.csv
Archivo leído correctamente: draft_combine_stats.csv
Archivo leído correctamente: game.csv
Archivo leído correctamente: inactive_players.csv
Archivo leído correctamente: other_stats.csv
Archivo leído correctamente: play_by_play.csv
Archivo leído correctamente: team.csv


### GAMES

In [24]:
# Vemos el head, los primeros 5
games.head()

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,2,0,Regular Season
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,...,NaN,NaN,NaN,NaN,NaN,25.0,51.0,-5,0,Regular Season
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,53.0,-6,0,Regular Season
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,22.0,47.0,-16,0,Regular Season
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,50.0,17,0,Regular Season


In [25]:
# Obteción de informacion osbre la tabla
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65698 entries, 0 to 65697
Data columns (total 55 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   season_id               65698 non-null  int64  
 1   team_id_home            65698 non-null  int64  
 2   team_abbreviation_home  65698 non-null  object 
 3   team_name_home          65698 non-null  object 
 4   game_id                 65698 non-null  int64  
 5   game_date               65698 non-null  object 
 6   matchup_home            65698 non-null  object 
 7   wl_home                 65696 non-null  object 
 8   min                     65698 non-null  int64  
 9   fgm_home                65685 non-null  float64
 10  fga_home                50251 non-null  float64
 11  fg_pct_home             50208 non-null  float64
 12  fg3m_home               52480 non-null  float64
 13  fg3a_home               47015 non-null  float64
 14  fg3_pct_home            46624 non-null

In [26]:
# Vemos las columnas
games.columns

Index(['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home',
       'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home',
       'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home',
       'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home',
       'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away',
       'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away',
       'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away',
       'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away',
       'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away',
       'season_type'],
      dtype='object')

In [27]:
# Filtramos las columnas a utilizar
games = games[['game_date', 'game_id', 'team_id_home', 'wl_home', 'wl_away', 'team_id_away', 'pts_home', 'pts_away']]

# Observamos
games.head()

,game_date,game_id,team_id_home,wl_home,wl_away,team_id_away,pts_home,pts_away
0,1946-11-01 00:00:00,24600001,1610610035,L,W,1610612752,66.0,68.0
1,1946-11-02 00:00:00,24600003,1610610034,W,L,1610610031,56.0,51.0
2,1946-11-02 00:00:00,24600002,1610610032,W,L,1610612738,59.0,53.0
3,1946-11-02 00:00:00,24600004,1610610025,W,L,1610612752,63.0,47.0
4,1946-11-02 00:00:00,24600005,1610610028,L,W,1610610036,33.0,50.0


In [28]:
# Vemos la información
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65698 entries, 0 to 65697
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   game_date     65698 non-null  object 
 1   game_id       65698 non-null  int64  
 2   team_id_home  65698 non-null  int64  
 3   wl_home       65696 non-null  object 
 4   wl_away       65696 non-null  object 
 5   team_id_away  65698 non-null  int64  
 6   pts_home      65698 non-null  float64
 7   pts_away      65698 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 4.0+ MB


In [29]:
# Colocamos el formato fecha y filtramos las fechas de interés
games['game_date'] = pd.to_datetime(games['game_date'])

# Filtrar por rango de años (2013 a 2023, inclusive)
games_table = games[(games['game_date'].dt.year >= 2013) & (games['game_date'].dt.year <= 2023)]

# Creamos columna año
games_table['year'] = games_table['game_date'].dt.year

# Observamos
games_table.sample(8)

,game_date,game_id,team_id_home,wl_home,wl_away,team_id_away,pts_home,pts_away,year
54867,2014-10-13,11400044,1610612741,W,L,1610612743,110.0,90.0,2014
53325,2014-04-09,21301169,1610612750,L,W,1610612741,87.0,102.0,2014
55066,2015-11-01,21500041,1610612748,W,L,1610612745,109.0,89.0,2015
61204,2020-01-18,21900631,1610612741,W,L,1610612739,118.0,116.0,2020
52790,2014-01-24,21300641,1610612741,L,W,1610612746,95.0,112.0,2014
57882,2017-10-20,21700026,1610612756,L,W,1610612747,130.0,132.0,2017
62006,2021-02-01,22000316,1610612749,W,L,1610612757,134.0,106.0,2021
64788,2022-12-22,22200477,1610612740,W,L,1610612759,126.0,117.0,2022


In [30]:
# Observamos la información
games_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13627 entries, 52071 to 65697
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   game_date     13627 non-null  datetime64[ns]
 1   game_id       13627 non-null  int64         
 2   team_id_home  13627 non-null  int64         
 3   wl_home       13627 non-null  object        
 4   wl_away       13627 non-null  object        
 5   team_id_away  13627 non-null  int64         
 6   pts_home      13627 non-null  float64       
 7   pts_away      13627 non-null  float64       
 8   year          13627 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(1), int64(3), object(2)
memory usage: 1011.4+ KB


In [31]:
games_table.to_csv("games_table", index=False)

## TEAMS

In [32]:
# Observamos
teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949.0
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946.0
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970.0
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002.0
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966.0


In [33]:
# Filtramos columnas
teams = teams[['id', 'city', 'nickname']]
teams_table = teams

teams_table.to_csv("teams_table", index=False)



In [34]:
teams_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        30 non-null     int64 
 1   city      30 non-null     object
 2   nickname  30 non-null     object
dtypes: int64(1), object(2)
memory usage: 852.0+ bytes


In [35]:
# Unimos tabla GAMES y TEAMS con equipos de CASA(local) mediante un JOIN
#home_games =  pd.merge(games_table, teams, left_on='team_id_home', right_on='id', how='inner')

# Renombramos columnas
#home_games = home_games.rename(columns={
#    'city': 'city_home',
#    'nickname': 'nickname_home'
#})

#home_games.head()

In [36]:
# # Combinamos las tablas de juegos de CASA y de VISITANTES
# both_games = pd.merge(home_games, teams, left_on='team_id_away', right_on='id', how='inner')

# # Renombramos columnas
# both_games = both_games.rename(columns={
#     'city': 'city_away',
#     'nickname': 'nickname_away'
# })

# # Eliminamos columnas
# both_games = both_games.drop(columns=[
#     'team_id_home', 'team_id_away', 'id_x', 'id_y'
# ])

# # Observamos
# both_games.head()

In [37]:
# # Combinamos las tablas de juegos de CASA y de VISITANTES
# both_games = pd.merge(home_games, teams, left_on='team_id_away', right_on='id', how='inner')

# # Renombramos columnas
# both_games = both_games.rename(columns={
#     'city': 'city_away',
#     'nickname': 'nickname_away'
# })

# # Eliminamos columnas
# both_games = both_games.drop(columns=[
#     'team_id_home', 'team_id_away', 'id_x', 'id_y'
# ])

# # Observamos
#both_games.head()

In [38]:
# Observamos la info
#both_games.info()

## Play by play

In [39]:
game_detail.head()

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,29600012,0,12,0,1,14:43 PM,12:00,NaN,Start of 1st Period (14:43 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,29600012,2,10,0,1,14:50 PM,12:00,Jump Ball O'Neal vs. Kleine: Tip to Cassell,NaN,NaN,...,Suns,PHX,5.0,208,Sam Cassell,1.610613e+09,Phoenix,Suns,PHX,0
2,29600012,3,2,1,1,14:51 PM,11:45,NaN,NaN,MISS Cassell 15' Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
3,29600012,4,4,0,1,14:51 PM,11:43,O'Neal REBOUND (Off:0 Def:1),NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,29600012,5,2,1,1,14:51 PM,11:29,MISS Ceballos 26' 3PT Jump Shot,NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0


In [40]:
#Seleccionamos las columnas con las que trabajaremos
game_detail = game_detail[['game_id', 'person1type', 'player1_name']]

#Se suman los valores duplicados
game_detail.duplicated().sum()

#Se eliminan valores duplicados
game_detail.drop_duplicates(inplace=True)

#Se eliminan valores nulos o vacios
game_detail.dropna(inplace=True)

#Se extrae informacion que tenga valores mayores a cero(0) en caso que aparezcan el el DF
game_detail = game_detail[game_detail["person1type"] > 0.0]

game_detail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 610035 entries, 1 to 13592369
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   game_id       610035 non-null  int64  
 1   person1type   610035 non-null  float64
 2   player1_name  610035 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 18.6+ MB


In [41]:
# games_info = pd.merge(games_1, game_detail,  left_on='game_id', right_on='game_id', how='inner')
# games_info.head()

# player_pts = games_info.groupby(['year', 'player1_name'])['person1type'].mean().reset_index().sort_values('year')
# player_pts.head(10)

# player_pts.info()

# player_pts[player_pts['year'] == 2022].sort_values(by='person1type', ascending=False).head(10)



In [42]:
game_detail_table = game_detail
game_detail_table.to_csv("game_detail_table", index=False)


## Jugadores Inactivos

In [43]:
inactive_players.head()

,game_id,player_id,first_name,last_name,jersey_num,team_id,team_city,team_name,team_abbreviation
0,29600034,184,Bobby,Phills,14.0,1610612739,Cleveland,Cavaliers,CLE
1,29600034,781,Will,Perdue,41.0,1610612759,San Antonio,Spurs,SAS
2,29600132,120,Steven,Smith,8.0,1610612737,Atlanta,Hawks,ATL
3,29600203,698,Eddie,Johnson,8.0,1610612754,Indiana,Pacers,IND
4,29600213,32,Clifford,Rozier,44.0,1610612753,Orlando,Magic,ORL


In [44]:
inactive_players['inactive_player_name'] = inactive_players['first_name'].astype(str) + ' ' + inactive_players['last_name'].astype(str)

inactive_players = inactive_players[['game_id', 'inactive_player_name']]

inactive_players_sin_duplicados = inactive_players.drop_duplicates(subset='inactive_player_name', keep='first')

inactive_players_sin_duplicados.reset_index(drop=True, inplace=True)

In [45]:
# # Hacemos merge
# games_final_info = pd.merge(games_info, inactive_players, left_on='game_id', right_on='game_id', how='inner')

# # Quitamos duplicados y nulos
# games_final_info.drop_duplicates(inplace=True)
# games_final_info.dropna(inplace=True)

# # Observamos
# games_final_info.head()

In [46]:
inactive_players.head()

,game_id,inactive_player_name
0,29600034,Bobby Phills
1,29600034,Will Perdue
2,29600132,Steven Smith
3,29600203,Eddie Johnson
4,29600213,Clifford Rozier


In [47]:
inactive_players_table = inactive_players
inactive_players_table.to_csv("inactive_players_table", index=False)

In [48]:
inactive_players_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110191 entries, 0 to 110190
Data columns (total 2 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   game_id               110191 non-null  int64 
 1   inactive_player_name  110191 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.7+ MB


In [49]:
# games_info = pd.merge(games_1, game_detail,  left_on='game_id', right_on='game_id', how='inner')
# games_info.head()

# player_pts = games_info.groupby(['year', 'player1_name'])['person1type'].mean().reset_index().sort_values('year')
# player_pts.head(10)

# player_pts.info()

# player_pts[player_pts['year'] == 2022].sort_values(by='person1type', ascending=False).head(10)

---


## Información común de los jugadores

In [50]:
info_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4171 entries, 0 to 4170
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   person_id                         4171 non-null   int64  
 1   first_name                        4171 non-null   object 
 2   last_name                         4171 non-null   object 
 3   display_first_last                4171 non-null   object 
 4   display_last_comma_first          4171 non-null   object 
 5   display_fi_last                   4171 non-null   object 
 6   player_slug                       4171 non-null   object 
 7   birthdate                         4171 non-null   object 
 8   school                            4156 non-null   object 
 9   country                           4170 non-null   object 
 10  last_affiliation                  4171 non-null   object 
 11  height                            4075 non-null   object 
 12  weight

In [51]:
info_players.head()

,person_id,first_name,last_name,display_first_last,display_last_comma_first,display_fi_last,player_slug,birthdate,school,country,...,playercode,from_year,to_year,dleague_flag,nba_flag,games_played_flag,draft_year,draft_round,draft_number,greatest_75_flag
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24 00:00:00,Duke,USA,...,HISTADD_alaa_abdelnaby,1990.0,1994.0,N,Y,Y,1990,1,25,N
1,76003,Kareem,Abdul-Jabbar,Kareem Abdul-Jabbar,"Abdul-Jabbar, Kareem",K. Abdul-Jabbar,kareem-abdul-jabbar,1947-04-16 00:00:00,UCLA,USA,...,HISTADD_kareem_abdul-jabbar,1969.0,1988.0,N,Y,Y,1969,1,1,Y
2,1505,Tariq,Abdul-Wahad,Tariq Abdul-Wahad,"Abdul-Wahad, Tariq",T. Abdul-Wahad,tariq-abdul-wahad,1974-11-03 00:00:00,San Jose State,France,...,tariq_abdul-wahad,1997.0,2003.0,N,Y,Y,1997,1,11,N
3,949,Shareef,Abdur-Rahim,Shareef Abdur-Rahim,"Abdur-Rahim, Shareef",S. Abdur-Rahim,shareef-abdur-rahim,1976-12-11 00:00:00,California,USA,...,shareef_abdur-rahim,1996.0,2007.0,N,Y,Y,1996,1,3,N
4,76005,Tom,Abernethy,Tom Abernethy,"Abernethy, Tom",T. Abernethy,tom-abernethy,1954-05-06 00:00:00,Indiana,USA,...,HISTADD_tom_abernethy,1976.0,1980.0,N,Y,Y,1976,3,43,N


In [52]:
info_players = info_players[['team_id', 'team_name', 'team_code', 'display_last_comma_first', 'birthdate', 'school', 'country', 'season_exp', 'position', 'draft_year', 'from_year', 'to_year']]

#Se suman los valores duplicados
info_players.duplicated().sum()

#Se eliminan valores duplicados
info_players.drop_duplicates(inplace=True)

#Se eliminan valores nulos o vacios
info_players.dropna(inplace=True)



info_players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3449 entries, 0 to 4170
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   team_id                   3449 non-null   int64  
 1   team_name                 3449 non-null   object 
 2   team_code                 3449 non-null   object 
 3   display_last_comma_first  3449 non-null   object 
 4   birthdate                 3449 non-null   object 
 5   school                    3449 non-null   object 
 6   country                   3449 non-null   object 
 7   season_exp                3449 non-null   float64
 8   position                  3449 non-null   object 
 9   draft_year                3449 non-null   object 
 10  from_year                 3449 non-null   float64
 11  to_year                   3449 non-null   float64
dtypes: float64(3), int64(1), object(8)
memory usage: 350.3+ KB


In [53]:
info_players.sample(10)

,team_id,team_name,team_code,display_last_comma_first,birthdate,school,country,season_exp,position,draft_year,from_year,to_year
3417,1610612763,Grizzlies,grizzlies,"Smart, Marcus",1994-03-06 00:00:00,Oklahoma State,USA,9.0,Guard,2014,2014.0,2023.0
2055,1610610028,Falcons,falcons,"King, Tom",1924-03-09 00:00:00,Michigan,USA,1.0,Guard,Undrafted,1946.0,1946.0
1142,1610612737,Hawks,hawks,"Ferrell, Duane",1965-02-28 00:00:00,Georgia Tech,USA,11.0,Forward,Undrafted,1988.0,1998.0
3793,1610612742,Mavericks,mavericks,"Vianna, Joao",1966-11-15 00:00:00,Travajara,Brazil,1.0,Forward,Undrafted,1991.0,1991.0
390,1610612751,Nets,nets,"Boynes, Winford",1957-05-17 00:00:00,San Francisco,USA,3.0,Guard,1978,1978.0,1980.0
3253,1610612738,Celtics,celtics,"Sanders, Frankie",1957-01-23 00:00:00,Southern,USA,2.0,Forward,1978,1978.0,1980.0
2971,1610612756,Suns,suns,"Plumlee, Miles",1988-09-01 00:00:00,Duke,USA,8.0,Center,2012,2012.0,2018.0
1307,1610612760,Thunder,thunder,"Gilgeous-Alexander, Shai",1998-07-12 00:00:00,Kentucky,Canada,5.0,Guard,2018,2018.0,2023.0
2530,1610610037,Hawks,hawks,"Miksis, Al",1928-02-02 00:00:00,Western Illinois,USA,1.0,Center,Undrafted,1949.0,1949.0
2161,1610612737,Hawks,hawks,"Law, Acie",1985-01-25 00:00:00,Texas A&M,USA,5.0,Guard,2007,2007.0,2010.0


In [54]:

info_players.to_csv("info_players", index=False)

---

## EDA (Análisis exploratorio de datos)

Preparacion de datos


In [30]:
# games_1 = both_games
# games_1.head()

# games_1['total_points'] = games_1['pts_home'] + games_1['pts_away']
# games_1.head()

# games_1.nlargest(n=5, columns='total_points')

# games_1.nsmallest(n=5, columns='total_points')

In [31]:
# total_points = games_1['wl_home'].value_counts()
# total_points

Cantidad de puntos por equipo cuando jueguen en casa o de visita

In [32]:
# plt.pie(total_points.values, labels=['Casa', 'Visitante'], autopct='%.0f%%')
# plt.show()

Graficacion de rendimiento 

In [33]:
# plt.figure(figsize=(20,6))
# home_wins = games_1[games_1['wl_home'] == 1]
# result = home_games.groupby('nickname_home')['pts_home'].mean().reset_index().sort_values('pts_home')


# ax = sns.barplot(data=home_games, x='nickname_home', y='pts_home', order=result['nickname_home'], errorbar=None)
# ax.bar_label(ax.containers[-1], fmt='%.0f', label_type='center', rotation=45)
# plt.xticks(rotation=75)  # Rotar etiquetas si es necesario
# plt.title("Puntos promedio por equipo en casa")
# plt.show()

# print(home_wins.columns)

Histograma del promedio de puntaje en los encuentros

In [34]:
# sns.histplot(data=games_1, x='total_points', bins=50)
# plt.title("Puntos promedio por encuentro")

Rango de puntos totales con variaciones en juegos menores

In [35]:
# sns.boxplot(x=games_1['total_points'])
# plt.title("Rango de puntos totales")

In [36]:
# print(home_wins.head())
# print(home_wins.shape)

Graciacion de partidos ganados por equipo en casa 

In [37]:
# plt.figure(figsize=(20,6))
# result = games_1['nickname_home'].value_counts().sort_values()

# ax = sns.countplot(x='nickname_home', data=games_1, order=result.index)
# ax.bar_label(ax.containers[-1], label_type='center', rotation=45)
# plt.xticks(rotation=75)
# plt.title("Partidos ganados por equipo en casa")
# plt.show()

In [38]:
# plt.figure(figsize=(20,6))
# result = games_1['nickname_away'].value_counts().sort_values()

# ax = sns.countplot(x='nickname_away', data=games_1, order=result.index)
# ax.bar_label(ax.containers[-1], label_type='center', rotation=45)
# plt.xticks(rotation=75)
# plt.title("Partidos ganados por equipo de visitante")
# plt.show()

In [39]:
# games_1.groupby('year')['total_points'].mean()

In [40]:
# plt.figure(figsize=(20,6))

# ax = sns.lineplot(data=games_1, x='year', y='total_points')
# plt.title("Linea de tiempo en puntuación en 10 años")

# plt.show()


Partidos de Casa vs Visita comparacion en puntos

In [41]:
# plt.figure(figsize=(20,6))
# x = games_1[['nickname_home', 'nickname_away']].value_counts()

# ax = x[:10].plot(kind="bar", figsize=[10, 5])
# ax.set_title('Casa vs Visita')
# ax.set_xlabel('Partidos')
# ax.set_ylabel('Cantidad')
# ax.set_xticklabels(x.index[:10], rotation = 60)
# #plt.title("Partidos ganados en casa por VS de equipos")

# for p in ax.patches:
#     ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width()/2, p.get_height()), ha='center', va='bottom')
# plt.show()



---

Los jugadores que tuvieron mejor puntuacion en el 2022

Agregacion de Tabla de Jugadores inactivos por encuentros

# Conexión con Base de SQL

In [42]:
# import pyodbc #importamos 

# # Parámetros de conexión
# server = "DESKTOP-UTQDVSQ\SQLEXPRESS"  # Nombre del servidor seleccionar la base de datos por el equipo
# database = "DATOFINALNBAcsv"            # Reemplaza con el nombre de tu base de datos

# # Establecemos la conexión
# try:
#     # Intentamos conectar
#     conexion = pyodbc.connect(
#         f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
#     )
#     print("Conexión exitosa")
    
#     # Aquí puedes añadir más operaciones como ejecutar consultas SQL
#     cursor = conexion.cursor()
#     cursor.execute("SELECT TOP 10 * FROM dbo.DATOFINALNBA ")  # Reemplaza 'TuTabla' por el nombre de una tabla existente
#     for row in cursor:
#         print(row)

# except pyodbc.Error as e:
#     # Manejo de errores
#     print("Error en la conexión:", e)

In [44]:
import pyodbc  # Importamos la librería

# Parámetros de conexión
server = "DESKTOP-2PTA5VC\SQLEXPRESS"  # Nombre del servidor
database = "DATOSFINALESNBA2023"            # Nombre de la base de datos

# Lista de tablas a consultar
tablas = ["game_detail_table", "games_table", "inactive_players_table", "teams_table"]

# Establecemos la conexión
try:
    # Intentamos conectar
    conexion = pyodbc.connect(
        f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
    )
    print("Conexión exitosa")

    # Cursor para ejecutar consultas
    cursor = conexion.cursor()

    # Iterar sobre las tablas
    for tabla in tablas:
        print(f"\nConsultando datos de la tabla: {tabla}")
        query = f"SELECT TOP 10 * FROM dbo.{tabla}"  # Consulta para cada tabla
        cursor.execute(query)
        
        # Imprimir resultados
        for row in cursor:
            print(row)

except pyodbc.Error as e:
    # Manejo de errores
    print("Error en la conexión:", e)

#En caso de no querer manipular los archivos se debe cerrar la conexión con la DB

#finally:
    # Cerrar conexión
#    if 'conexion' in locals():
#        conexion.close()
#        print("\nConexión cerrada")


Conexión exitosa

Consultando datos de la tabla: game_detail_table
('29600012', '4.0', "Shaquille O'Neal")
('29600012', '5.0', 'Sam Cassell')
('29600012', '4.0', 'Cedric Ceballos')
('29600012', '4.0', 'Nick Van Exel')
('29600012', '5.0', 'Robert Horry')
('29600012', '5.0', 'Joe Kleine')
('29600012', '5.0', 'Wesley Person')
('29600012', '4.0', 'Eddie Jones')
('29600012', '4.0', 'Elden Campbell')
('29600012', '4.0', 'Derek Fisher')

Consultando datos de la tabla: games_table
('2013-04-20', '41200161', '1610612743', 'W', 'L', '1610612744', '97.0', '95.0', '2013')
('2013-04-20', '41200131', '1610612751', 'W', 'L', '1610612741', '106.0', '89.0', '2013')
('2013-04-20', '41200111', '1610612752', 'W', 'L', '1610612738', '85.0', '78.0', '2013')
('2013-04-20', '41200171', '1610612746', 'W', 'L', '1610612763', '112.0', '91.0', '2013')
('2013-04-21', '41200121', '1610612754', 'W', 'L', '1610612737', '107.0', '90.0', '2013')
('2013-04-21', '41200141', '1610612760', 'W', 'L', '1610612745', '120.0', 